In [2]:
#import usual libraries
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
from numpy import linalg as LA
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [3]:
#list of powers of 2 in decreasing order starting at 2**nqbits-1
def pow2(nqbits):
    return (2**(nqbits-1-np.arange(nqbits)))

In [4]:
pow2(2)

array([2, 1])

In [7]:
#binary decompositions with nqbits digits
def bindigits(integer,nqbits):
    yy=np.zeros(nqbits,dtype=np.int32) #binary decomposition of integer with nqbit digits
    for ff in range(nqbits):
        yy[ff]=integer//(2**(nqbits-1-ff))
        print(yy[ff])
        integer=integer%(2**(nqbits-1-ff))
        #print(integer)
        pass
    return yy

In [8]:
bindigits(15,4)

1
1
1
1


array([1, 1, 1, 1], dtype=int32)

In [9]:
#making the sigma matrices
#at this point the number of qubits nq is fixed
nq=2
mmax=2**nq
#initializing sigma_x at ns locations matrix to 0
#sigmax=np.zeros((ns,mmax,mmax))
sigmax=np.zeros((nq,mmax,mmax))
#initializing sigma_z at ns locations matrix to 0
sigmaz=np.zeros((nq,mmax,mmax))

ident=np.zeros((mmax,mmax))
for mm in range(mmax):
    ident[mm,mm]=1.0
    yy=bindigits(mm,nq)
    for hh in range(nq):
        flyy=np.array(yy)
        flyy[hh]=(yy[hh]+1)%2
        reconfl=np.inner(flyy,pow2(nq))
        sigmax[hh,reconfl,mm]=1
        sigmaz[hh,mm,mm]=(1-2*yy[hh])
        pass

0
0
0
1
1
0
1
1


In [14]:
np.allclose(sigmax[0],np.kron(np.array([[0,1],[1,0]]),np.eye(2) ) )
np.allclose(sigmaz[0],np.kron(np.array([[1,0],[0,-1]]),np.eye(2) ) )
np.allclose(sigmax[1],np.kron(np.eye(2),np.array([[0,1],[1,0]]) ))
np.allclose(sigmaz[1],np.kron(np.eye(2),np.array([[1,0],[0,-1]]) ) )
bindigits(7,3)

True

True

True

True

1
1
1


array([1, 1, 1], dtype=int32)

In [8]:
##### Test1
sig_z=np.array([[1,0],[0,-1]])
iden=np.array([[1,0],[0,1]])
sig_x=np.array([[0,1],[1,0]])

zero_1qubit_state=np.array([[1],[0]])
one_1qubit_state=np.array([[0],[1]])

zero_2Q=np.tensordot(zero_1qubit_state,zero_1qubit_state,axes=0).reshape((4,1))
one_2Q=np.tensordot(zero_1qubit_state,one_1qubit_state,axes=0).reshape((4,1))
two_2Q=np.tensordot(one_1qubit_state,zero_1qubit_state,axes=0).reshape((4,1))
three_2Q=np.tensordot(one_1qubit_state,one_1qubit_state,axes=0).reshape((4,1))

np.kron(iden,sig_x)

array([[0, 1, 0, 0],
       [1, 0, 0, 0],
       [0, 0, 0, 1],
       [0, 0, 1, 0]])

In [ ]:
max(np.abs((np.reshape(np.matmul(sigmax[3],sigmax[3])-ident,mmax*mmax))))

In [ ]:
max(np.abs(np.reshape(np.matmul(sigmax[3],sigmaz[3])+np.matmul(sigmaz[3],sigmax[3]),mmax*mmax)))

In [ ]:
max(np.abs(np.reshape(np.matmul(sigmax[3],sigmaz[2])-np.matmul(sigmaz[2],sigmax[3]),mmax*mmax)))

In [17]:
#making the general hamiltonian    
def ham(lamb1,lamb2,bc):
    #lamb1=0.1 nearest neighbor
    #lamb2=1.0  #onsite
    #bc==1: PBC
    #initializing the energy matrix to 0
    hammatr=np.zeros((mmax,mmax))
    for nn in range(nq-1):
        hammatr+=-lamb2*sigmaz[nn]-lamb1*np.matmul(sigmax[nn],sigmax[nn+1])
        pass
    hammatr+=-lamb2*sigmaz[nq-1]
    if bc==1:
        hammatr+=-lamb1*np.matmul(sigmax[nq-1],sigmax[0]) 
    return hammatr 

In [ ]:
def four_fermi_Hamiltonian(G,nq):
    dim=2**nq
    dim2=4**nq
    
    H1=np.zeros((dim,dim)).astype(np.complex_)
    H2=np.zeros((dim,dim)).astype(np.complex_)
    H3=np.zeros((dim2,dim2)).astype(np.complex_)
    
    for i in range(nq-1):
        H1 += (sigma_creator(nq,i,0) @ sigma_creator(nq,i+1,0))
        H2 += (sigma_creator(nq,i,1) @ sigma_creator(nq,i+1,1))
        H3 += 0.25*G**2 *np.kron ((np.identity(dim)+ sigma_creator(nq,i,2)), (np.identity(dim)+sigma_creator(nq,i,2)) )  
    
    #print(np.shape(H1))
    #print(np.shape(H2))
    #print(np.shape(H3))
    H=np.kron(H1,H1)+np.kron(H2,H2)+H3
    return H

#four_fermi_Hamiltonian(0.1,4)

# eigen value
G=0.1
nq=4
mmax=4**nq
four_fermi_eigenvalues, four_fermi_eigenvectors=LA.eigh(four_fermi_Hamiltonian(G,nq))
four_fermi_eigenvalues

plt.plot(four_fermi_eigenvalues,'x')

In [24]:
#site occupations
nnocc=np.zeros((nq,mmax,mmax))
for nn in range(nq):
    nnocc[nn]=0.5*(ident-sigmaz[nn])
    pass
totalnn=(nnocc[0]+nnocc[1]+nnocc[2]+nnocc[3])    
nnocc[3]
totalnn

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 2., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 2., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 2., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 3., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 2., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 2., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 3., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 2., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,

In [ ]:
np.matmul(nnocc[1],nnocc[2])

In [ ]:
#ei=LA.eigh(ham(0.02,1.0,0))
ei=LA.eigh(ham(G,nq) )
ei
mmax=4**nq
#evolution with correct complex conjugation    
def evol(tt):
    proj=np.zeros((mmax,mmax))+0.0+0.0j
    for uu in range(mmax):
        proj+=np.exp(-tt*1.0j*ei[0][uu])*np.outer(ei[1][:,uu],np.conjugate(ei[1][:,uu]))
        pass
    return proj
instate=np.zeros(mmax)+0.0j 
#|1000> state
instate[7]=1.0
# number of time steps
nstep=4000
#time step
delt=0.001
times1=np.zeros(nstep)
meas0=np.zeros(nstep)+0.0j  
meas1=np.zeros(nstep)+0.0j  
#going through all the time steps
for nst in range(nstep):
    #time
    tt=nst*delt
    times1[nst]=tt
    #evolution at time tt on instate
    wavet=np.dot(evol(tt),instate)
    #hermitian conjugate
    instatecc=np.conj(np.transpose(instate))
    
   # projstate0=np.dot(nnocc[0],wavet)
   # projstate1=np.dot(nnocc[1],wavet)
    meas0[nst]=(np.abs(np.inner(instatecc,wavet)))**2
    #meas1[nst]=np.inner(instatecc,projstate1)
plt.plot(times1,np.real(meas0),'.',ls='none')    
#plt.plot(times1,np.real(meas1))
plt.show()



In [ ]:
#Trotter version


ei1=LA.eigh(ham(0.02,0.0,0))
#evolution with correct complex conjugation    
def evol1(tt):
    proj=np.zeros((mmax,mmax))+0.0+0.0j
    for uu in range(mmax):
        proj+=np.exp(-tt*1.0j*ei1[0][uu])*np.outer(ei1[1][:,uu],np.conjugate(ei1[1][:,uu]))
        pass
    return proj

print(ei1[0][0:mmax])#energy eigenvalues

ei2=LA.eigh(ham(0.0,1.0,0))
#evolution with correct complex conjugation    
def evol2(tt):
    proj=np.zeros((mmax,mmax))+0.0+0.0j
    for uu in range(mmax):
        proj+=np.exp(-tt*1.0j*ei2[0][uu])*np.outer(ei2[1][:,uu],np.conjugate(ei2[1][:,uu]))
        pass
    return proj
print(ei2[0][0:mmax])#energy eigenvalues

In [ ]:
#Trotter version
deltat=10
# number of time steps
nstep2=25

ei1=LA.eigh(ham(0.02,0.0,0))
#evolution with INcorrect complex conjugation    
def evol1(tt):
    proj=np.zeros((mmax,mmax))+0.0+0.0j
    for uu in range(mmax):
        proj+=np.exp(-tt*1.0j*ei1[0][uu])*np.outer(np.conjugate(ei1[1][:,uu]),ei1[1][:,uu])
        pass
    return proj
ei2=LA.eigh(ham(0.0,1.0,0))
#evolution with INcorrect complex conjugation    
def evol2(tt):
    proj=np.zeros((mmax,mmax))+0.0+0.0j
    for uu in range(mmax):
        proj+=np.exp(-tt*1.0j*ei2[0][uu])*np.outer(np.conjugate(ei2[1][:,uu]),ei2[1][:,uu])
        pass
    return proj
trotstep=np.matmul(evol1(deltat),evol2(deltat))
times2=np.zeros(nstep2)
meas2=np.zeros(nstep2)+0.0j  
#going through all the Trotter steps
wavet=instate
for nst in range(nstep2):
    #time
    tt=nst*deltat
    times2[nst]=tt
    instatecc=np.conj(np.transpose(wavet))
    projstate=np.dot(nnocc[0],wavet)
    meas2[nst]=np.inner(instatecc,projstate)
    
    #evolution at time tt on instate
    wavet=np.dot(trotstep,wavet)

#plt.figure(figsize=(2.5,3))
plt.plot(times1,np.real(meas0))
plt.plot(times2,np.real(meas2))
#plt.plot(times2,np.real(meas2-meas1))
#plt.savefig("temp3.pdf")
plt.show()